In [1]:
from huggingface_hub import notebook_login
notebook_login()


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import sys


Mounted at /content/drive


In [3]:
!pip install datasets tokenizers

# Huggingface
from transformers import BertTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


In [4]:
!cp /content/drive/MyDrive/py_files/PositiveGenerator.py /content
!cp /content/drive/MyDrive/py_files/configuration.py /content
!cp /content/drive/MyDrive/py_files/ToxicDetector.py /content

In [5]:
# models
import PositiveGenerator as pg
import ToxicDetector as td
import configuration as cfg

import os
import pathlib

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision

# General
from tqdm import tqdm
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Device Setting
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("GPU")
else:
  device = torch.device("cpu")
  print("CPU")

GPU


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size

detector_config = cfg.get_config(num_epochs = 1, d_model= 256, h=4, depth=2, vocab_size = vocab_size)
generator_config = cfg.get_config(num_epochs = 1, d_model= 512, h=32, depth=3, vocab_size = vocab_size)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# DUAL MODEL

In [42]:
class DetectingTransformer(nn.Module):
  def __init__(self, detector_config, generator_config, vocab_size, tokenizer):
    super(DetectingTransformer, self).__init__()

    # Toxic Detector
    self.td_model = td.ToxicDetector(detector_config, vocab_size).to(device)
    self.td_model.load_state_dict(torch.load(f'td_final_model_2_4_256.pth',map_location=device))

    # Positive Generator
    self.pg_model = pg.PositiveGenerator(generator_config, vocab_size).to(device)
    self.pg_model.load_state_dict(torch.load(f'pg_final_model_3_32_512.pth',map_location=device))

    # Tokenizer
    self.tokenizer = tokenizer

    # Hyperparameters
    self.detector_h = detector_config['h']
    self.generator_h = generator_config['h']
    self.seq_len = detector_config['seq_len']

  def forward(self, x):
    detector_h = self.detector_h
    generator_h = self.generator_h
    seq_len = self.seq_len

    # Tokenizing
    tokenized_source = self.tokenizer(x, max_length = self.seq_len, padding = 'max_length', truncation=True, return_tensors = 'pt')
    # Input & mask
    encoder_input = torch.tensor(tokenized_source['input_ids'], dtype=torch.long).to(device)
    # source mask
    encoder_mask = torch.tensor(tokenized_source['attention_mask'], dtype= torch.long).unsqueeze(0).to(device)
    encoder_mask = encoder_mask.repeat(1, detector_h, 1)
    encoder_mask = encoder_mask.expand(seq_len, detector_h, seq_len)
    encoder_mask = encoder_mask.transpose(0,1).contiguous()

    td_out = self.td_model(encoder_input, encoder_mask)
    _, pred = torch.max(td_out, dim=1)
    out = x
    if pred == 0:
      return f'Non-toxic comments: {out}'

    else:
      # source mask (generator has different structure)
      encoder_mask = torch.tensor(tokenized_source['attention_mask'], dtype= torch.long).unsqueeze(0).to(device)
      encoder_mask = encoder_mask.repeat(1, generator_h, 1)
      encoder_mask = encoder_mask.expand(seq_len, generator_h, seq_len)
      encoder_mask = encoder_mask.transpose(0,1).contiguous()
      encoder_mask = encoder_mask.unsqueeze(0)

      encoder_output = self.pg_model.encode(encoder_input, encoder_mask)
      decoder_input = torch.empty(1,1).fill_(101).type_as(encoder_input).to(device)

      while decoder_input.size(1) < seq_len:
        decoder_mask = torch.tril(torch.ones(generator_h, decoder_input.size(1), decoder_input.size(1))).unsqueeze(0).type(torch.int).to(device)
        decoder_out = self.pg_model.decode(decoder_input, encoder_output, encoder_mask, decoder_mask)
        prob = self.pg_model(decoder_out[:,-1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, torch.empty(1, 1).type_as(encoder_input).fill_(next_word.item()).to(device)], dim=1)

        if next_word == 102:
          break

      out = [tokenizer.decode(ids) for ids in decoder_input.tolist()]
      out = str(out).replace("[CLS]", "").replace("[SEP]", "").replace("[", "").replace("]", "")
    return f' Toxic detected. Reframed: {out}'


detecting_transformer = DetectingTransformer(detector_config, generator_config, vocab_size, tokenizer)
detecting_transformer.eval()

DetectingTransformer(
  (td_model): ToxicDetector(
    (emb): InputEmbedding(
      (embedding): Embedding(30522, 256)
    )
    (pe): PositionalEmbedding()
    (Encoder): Sequential(
      (0): EncoderBlock(
        (MultiHeadAttention): MultiHeadAttention(
          (W_q): Linear(in_features=256, out_features=256, bias=False)
          (W_k): Linear(in_features=256, out_features=256, bias=False)
          (W_v): Linear(in_features=256, out_features=256, bias=False)
          (W_o): Linear(in_features=256, out_features=256, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ln_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (FeedForward): FeedForward(
          (ffn): Sequential(
            (0): Linear(in_features=256, out_features=1024, bias=True)
            (1): ReLU()
            (2): Linear(in_features=1024, out_features=256, bias=True)
            (3): Dropout(

In [43]:
#@title Leaves a comment. It will automatistop editting to make incorrect information on wikipedia!!!cally transform if it has toxic contents in it

comment = "stop editting to make incorrect information on wikipedia!!! you *****" #@param {type: "string"}
out = detecting_transformer(comment)

print(out)

 Toxic detected. Reframed: ' please consider removing the guidelines on wikipedia by following the guidelines of the guidelines. thank you for your understanding. '


In [44]:
#@title Non-toxic example

comment = "stop editting to make incorrect information on wikipedia!!!" #@param {type: "string"}
out = detecting_transformer(comment)

print(out)

Non-toxic comments: stop editting to make incorrect information on wikipedia!!!
